In [ ]:
!pip install selenium
!pip install bs4
!pip install webdriver_manager

In [3]:
import webdriver_manager 
from selenium import webdriver
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3
import time
from bs4 import BeautifulSoup
import csv

In [4]:
urlFile = pd.read_csv("AmazonDot_urls.txt" , sep=",")
# urlFile.drop_duplicates(inplace = True)
all_links = urlFile['Link']

In [5]:
len(all_links)

75

### creating base csv file

In [4]:
#adding column names
try:
    with open("Amazon_product_details.csv", 'x', encoding='utf8', newline='') as csvFile:
        theWriter = csv.writer(csvFile)
        columns = ["Website", "Product_Link", "Product_Name", "Product_Brand", "Price",
                   "MRP", "Primary_Image_Links", "Product_Category","Gender"]
        theWriter.writerow(columns)
except FileExistsError as e:
    print("File exists Alreaady: 'Amazon_product_details.csv'")

In [10]:
start = 5
end = 15

page = 2

driver = webdriver.Chrome(ChromeDriverManager().install())
begin = time.time()
all_link_list = list(all_links)
all_link_list = all_link_list[start:end]
all_url_links_counts = start
print("Total Categories to scrape:-", len(all_link_list))
print()
with open("Amazon_product_details.csv", 'a', encoding='utf8', newline='') as csvFile:
    theWriter = csv.writer(csvFile)
    
    for cat_url in all_link_list:
        #adding page numbers to each category link
        final_url = []
        for i in range(page, 400):
            final_url.append(cat_url.format(page_no=i))
        print("Scraping for catogory No.{} ==> {}".format(all_url_links_counts,urlFile.Category[all_url_links_counts]))
        print('Page number....')
        pages = page-2
        for url in final_url:
            pages+=1
            driver.get(url)
            driver.implicitly_wait(50)
            soup=BeautifulSoup(driver.page_source,'html.parser')
            cards = soup.findAll('div', {'class':'a-section a-spacing-medium'})
            for card in cards:
                try:    
                    imgLink = card.findAll('img', {'class':'s-image'})[0]['src']
    #                     print(imgLink)
                    plink = card.findAll('a', {'class':'a-link-normal s-no-outline'})
                    proLink = "https://www.amazon.com/"+plink[0]['href']
    #                     print(proLink)
                    title = card.find('a', {'class':'a-link-normal a-text-normal'}).text.strip()
    #                     print(title)
                    brand = title.split(' ')[0]
    #                     print(brand)
                    try:
                        price = card.find('span', {'class':'a-offscreen'}).text.strip()
                    except:
                        price = np.nan
#                     print(price)
                    try:
                        mrp = card.findAll('span', {'class':'a-offscreen'})[1].text.strip()
                    except:
                        mrp = price
#                     print(mrp)
                    theWriter.writerow(["https://www.amazon.com/", proLink, title, brand, price, mrp, imgLink, urlFile.Category[all_url_links_counts],urlFile.Gender[all_url_links_counts]])
    
                except Exception as e:
                    print(e)
                    
            print(pages, end='->')
            
        all_url_links_counts+=1
        page = 2
        print('\n')
print(f"Completed! Total time required (minutes):{round(((time.time()-begin)/60), 2)} minutes")

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\DELL\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache


Total Categories to scrape:- 10

Scraping for catogory No.5 ==> Women-Polos
Page number....
396->397->398->399->400->

Scraping for catogory No.6 ==> Women-Tanks & Camis
Page number....
3->4->

KeyboardInterrupt: 

# Creating Dataframe

In [11]:
product_details_sel = pd.read_csv('Amazon_product_details.csv')
product_details_sel.drop_duplicates(subset = 'Primary_Image_Links',inplace = True)
product_details_sel.dropna(inplace = True)
product_details_sel.reset_index(drop = True, inplace = True)

In [12]:
product_details_sel["Price"] = [i.replace("$","") for i in product_details_sel["Price"]]
product_details_sel["MRP"] = [i.replace("$","") for i in product_details_sel["MRP"]]
product_details_sel["Primary_Image_Links"] = [i.replace("UL320_","UL960_FMwebp_QL65_") for i in product_details_sel["Primary_Image_Links"]]
product_details_sel['Price'] = product_details_sel['Price'].astype('float')
product_details_sel['MRP'] = product_details_sel['MRP'].astype('float')

In [14]:
product_details_sel.shape

(1179, 9)

# DataBase

In [27]:
conn = sqlite3.connect('amazonDbDotCom.db')
c = conn.cursor()

In [28]:
c.execute('CREATE TABLE IF NOT EXISTS product_details (Website varchar(40) , Product_Link TEXT PRIMARY KEY,Product_Name varchar(50) NOT NULL,Product_Brand varchar(50) NOT NULL,Product_Category varchar(50),Size_Avail varchar(20) ,Price REAL NOT NULL,MRP REAL NOT NULL,Gender varchar(15) ,Description TEXT ,Primary_Image_Links TEXT NOT NULL,Secondary_Image_Links TEXT ,Affiliate_Link TEXT)')
conn.commit()

In [29]:
product_details_sel.to_sql('product_details', conn, if_exists='replace', index = False)

In [30]:
# Print 20 products
c.execute('''  
SELECT * FROM product_details
          ''')

for row in c.fetchmany(size=5):
    print (row)

('https://www.amazon.com/', 'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_browse_fashion-womens-clothing_sr_pg395_1?ie=UTF8&adId=A093400939NGBJMYUY6PA&url=%2FSleeve-Collared-Athletic-Outdoor-Workout%2Fdp%2FB097M6VKBK%2Fref%3Dsr_1_18913_sspa%3Fqid%3D1637138437%26rnid%3D2368343011%26s%3Dapparel%26sr%3D1-18913-spons%26psc%3D1&qualifier=1637138437&id=3729961657726961&widgetName=sp_atf_browse', "JACK SMITH Women's Sleeveless Polo Shirts Quick Dry Golf Tennis Shirt Color Block Athletic Tops", 'JACK', 22.99, 22.99, 'https://m.media-amazon.com/images/I/51Qwv1919PS._AC_UL960_FMwebp_QL65_.jpg', 'Women-Polos', 'Women')
('https://www.amazon.com/', 'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_browse_fashion-womens-clothing_sr_pg395_1?ie=UTF8&adId=A05277171NE2TLR1Y9Y9&url=%2FBUBBLELIME-Womens-Sleeveless-Neck_White-S_BWST019%2Fdp%2FB08C9LP2W5%2Fref%3Dsr_1_18914_sspa%3Fqid%3D1637138437%26rnid%3D2368343011%26s%3Dapparel%26sr%3D1-18914-spons%26psc

In [31]:
conn.close()